In [1]:
using Rocket
using ReactiveMP
using GraphPPL
using BenchmarkTools
using Distributions
using MacroTools

┌ Info: Precompiling Rocket [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1317
┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1317


In [2]:
@model function smoothing(n, x0, c::ConstVariable, P::ConstVariable)
    
    x_prior ~ NormalMeanVariance(mean(x0), cov(x0)) 

    x = randomvar(n)
    y = datavar(Float64, n)
    
    x_prev = x_prior
    
    for i in 1:n
        x[i] ~ x_prev + c
        y[i] ~ NormalMeanVariance(x[i], P)
        
        x_prev = x[i]
    end

    return x, y
end

smoothing (generic function with 1 method)

In [3]:
P = 1.0
n = 500

data = collect(1:n) + rand(Normal(0.0, sqrt(P)), n);

In [4]:
function inference(data, x0, P)
    n = length(data)
    
    _, (x, y) = smoothing(n, x0, 1.0, P);

    x_buffer  = buffer(Marginal, n)
    marginals = getmarginals(x)
    
    subscription = subscribe!(marginals, x_buffer)
    
    update!(y, data)
    
    unsubscribe!(subscription)
    
    return getvalues(x_buffer)
end

inference (generic function with 1 method)

In [5]:
x0_prior = NormalMeanVariance(0.0, 10000.0)
@benchmark res = inference($data, $x0_prior, $P)

BenchmarkTools.Trial: 
  memory estimate:  14.32 MiB
  allocs estimate:  220096
  --------------
  minimum time:     18.368 ms (0.00% GC)
  median time:      19.704 ms (0.00% GC)
  mean time:        22.692 ms (13.64% GC)
  maximum time:     35.439 ms (40.36% GC)
  --------------
  samples:          221
  evals/sample:     1

In [6]:
inference(data, x0_prior, P)

500-element Vector{Marginal}:
 Marginal(NormalMeanVariance{Float64}(μ=1.118460388024951, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=2.118460388024952, v=0.0019999996000000823))
 Marginal(NormalMeanVariance{Float64}(μ=3.118460388024951, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=4.11846038802495, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=5.118460388024951, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=6.1184603880249515, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=7.118460388024952, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=8.118460388024953, v=0.001999999600000082))
 Marginal(NormalMeanVariance{Float64}(μ=9.11846038802495, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=10.118460388024952, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ=11.11846038802495, v=0.0019999996000000814))
 Marginal(NormalMeanVariance{Float64}(μ